Load dataset


In [ ]:
import multiprocessing as mp

mp.set_start_method("spawn", force=True)

In [ ]:
from huggingface_hub import login
import os


login(os.environ.get("HF_TOKEN"))

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mousserlane/id_receipt_dataset")

In [ ]:
dataset

In [ ]:
example = dataset["train"][0]
image = example["image"]
# let's make the image a bit smaller when visualizing
width, height = image.size
display(image.resize((int(width * 0.2), int(height * 0.2))))

In [ ]:
# let's load the corresponding JSON dictionary (as string representation)
ground_truth = example["ground_truth"]
print(ground_truth)

In [ ]:
from ast import literal_eval

literal_eval(ground_truth)["gt_parse"]

Load model and processor


In [ ]:
from transformers import VisionEncoderDecoderConfig

max_length = 768
image_size = [1280, 960]

# update image_size of the encoder
# during pre-training, a larger image size was used
config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")
config.encoder.image_size = image_size  # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length
# TODO we should actually update max_position_embeddings and interpolate the pre-trained ones:
# https://github.com/clovaai/donut/blob/0acc65a85d140852b8d9928565f0f6b2d98dc088/donut/model.py#L602

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel, BartConfig

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained(
    "naver-clova-ix/donut-base", config=config
)

In [ ]:
# import json
# import random
# from typing import Any, List, Tuple

# import torch
# from torch.utils.data import Dataset

# added_tokens = []


# class DonutDataset(Dataset):
#     """
#     DonutDataset which is saved in huggingface datasets format. (see details in https://huggingface.co/docs/datasets)
#     Each row, consists of image path(png/jpg/jpeg) and gt data (json/jsonl/txt),
#     and it will be converted into input_tensor(vectorized image) and input_ids(tokenized string).
#     Args:
#         dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
#         max_length: the max number of tokens for the target sequences
#         split: whether to load "train", "validation" or "test" split
#         ignore_id: ignore_index for torch.nn.CrossEntropyLoss
#         task_start_token: the special token to be fed to the decoder to conduct the target task
#         prompt_end_token: the special token at the end of the sequences
#         sort_json_key: whether or not to sort the JSON keys
#     """

#     def __init__(
#         self,
#         dataset_name_or_path: str,
#         max_length: int,
#         split: str = "train",
#         ignore_id: int = -100,
#         task_start_token: str = "<s>",
#         prompt_end_token: str = None,
#         sort_json_key: bool = True,
#     ):
#         super().__init__()

#         self.max_length = max_length
#         self.split = split
#         self.ignore_id = ignore_id
#         self.task_start_token = task_start_token
#         self.prompt_end_token = (
#             prompt_end_token if prompt_end_token else task_start_token
#         )
#         self.sort_json_key = sort_json_key

#         self.dataset = load_dataset(dataset_name_or_path, split=self.split)
#         self.dataset_length = len(self.dataset)

#         self.gt_token_sequences = []
#         for sample in self.dataset:
#             ground_truth = json.loads(sample["ground_truth"])
#             if (
#                 "gt_parses" in ground_truth
#             ):  # when multiple ground truths are available, e.g., docvqa
#                 assert isinstance(ground_truth["gt_parses"], list)
#                 gt_jsons = ground_truth["gt_parses"]
#             else:
#                 assert "gt_parse" in ground_truth and isinstance(
#                     ground_truth["gt_parse"], dict
#                 )
#                 gt_jsons = [ground_truth["gt_parse"]]

#             self.gt_token_sequences.append(
#                 [
#                     self.json2token(
#                         gt_json,
#                         update_special_tokens_for_json_key=self.split == "train",
#                         sort_json_key=self.sort_json_key,
#                     )
#                     + processor.tokenizer.eos_token
#                     for gt_json in gt_jsons  # load json from list of json
#                 ]
#             )

#         self.add_tokens([self.task_start_token, self.prompt_end_token])
#         self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(
#             self.prompt_end_token
#         )

#     def json2token(
#         self,
#         obj: Any,
#         update_special_tokens_for_json_key: bool = True,
#         sort_json_key: bool = True,
#     ):
#         """
#         Convert an ordered JSON object into a token sequence
#         """
#         if type(obj) == dict:
#             if len(obj) == 1 and "text_sequence" in obj:
#                 return obj["text_sequence"]
#             else:
#                 output = ""
#                 if sort_json_key:
#                     keys = sorted(obj.keys(), reverse=True)
#                 else:
#                     keys = obj.keys()
#                 for k in keys:
#                     if update_special_tokens_for_json_key:
#                         self.add_tokens([rf"<s_{k}>", rf"</s_{k}>"])
#                     output += (
#                         rf"<s_{k}>"
#                         + self.json2token(
#                             obj[k], update_special_tokens_for_json_key, sort_json_key
#                         )
#                         + rf"</s_{k}>"
#                     )
#                 return output
#         elif type(obj) == list:
#             return r"<sep/>".join(
#                 [
#                     self.json2token(
#                         item, update_special_tokens_for_json_key, sort_json_key
#                     )
#                     for item in obj
#                 ]
#             )
#         else:
#             obj = str(obj)
#             if f"<{obj}/>" in added_tokens:
#                 obj = f"<{obj}/>"  # for categorical special tokens
#             return obj

#     def add_tokens(self, list_of_tokens: List[str]):
#         """
#         Add special tokens to tokenizer and resize the token embeddings of the decoder
#         """
#         newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
#         if newly_added_num > 0:
#             model.decoder.resize_token_embeddings(len(processor.tokenizer))
#             added_tokens.extend(list_of_tokens)

#     def __len__(self) -> int:
#         return self.dataset_length

#     def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
#         """
#         Load image from image_path of given dataset_path and convert into input_tensor and labels
#         Convert gt data into input_ids (tokenized string)
#         Returns:
#             input_tensor : preprocessed image
#             input_ids : tokenized gt_data
#             labels : masked labels (model doesn't need to predict prompt and pad token)
#         """
#         sample = self.dataset[idx]

#         # inputs
#         pixel_values = processor(
#             sample["image"], random_padding=self.split == "train", return_tensors="pt"
#         ).pixel_values
#         pixel_values = pixel_values.squeeze()

#         # targets
#         target_sequence = random.choice(
#             self.gt_token_sequences[idx]
#         )  # can be more than one, e.g., DocVQA Task 1
#         input_ids = processor.tokenizer(
#             target_sequence,
#             add_special_tokens=False,
#             max_length=self.max_length,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )["input_ids"].squeeze(0)

#         labels = input_ids.clone()
#         labels[labels == processor.tokenizer.pad_token_id] = (
#             self.ignore_id
#         )  # model doesn't need to predict pad token
#         # labels[: torch.nonzero(labels == self.prompt_end_token_id).sum() + 1] = self.ignore_id  # model doesn't need to predict prompt (for VQA)
#         return pixel_values, labels, target_sequence

In [ ]:
# we update some settings which differ from pretraining; namely the size of the images + no rotation required
# source: https://github.com/clovaai/donut/blob/master/config/train_cord.yaml
processor.feature_extractor.size = image_size[::-1]  # should be (width, height)
processor.feature_extractor.do_align_long_axis = False
import os
from donut_dataset import DonutDataset

# os.environ["TOKENIZERS_PARALLELISM"] = "false"
train_dataset = DonutDataset(
    "mousserlane/id_receipt_dataset",
    max_length=max_length,
    split="train",
    task_start_token="<s_receipt-v1>",
    prompt_end_token="<s_receipt-v1>",
    sort_json_key=False,  # cord dataset is preprocessed, so no need for this
    processor=processor,
    model=model,
)
val_dataset = DonutDataset(
    "mousserlane/id_receipt_dataset",
    max_length=max_length,
    split="validation",
    task_start_token="<s_receipt-v1>",
    prompt_end_token="<s_receipt-v1>",
    sort_json_key=False,  # cord dataset is preprocessed, so no need for this
    processor=processor,
    model=model,
)
# if __name__ == "__main__":
#     train_dataset = DonutDataset(
#         "mousserlane/id_receipt_dataset",
#         max_length=max_length,
#         split="train",
#         task_start_token="<s_receipt-v1>",
#         prompt_end_token="<s_receipt-v1>",
#         sort_json_key=False,  # cord dataset is preprocessed, so no need for this
#     )

#     val_dataset = DonutDataset(
#         "mousserlane/id_receipt_dataset",
#         max_length=max_length,
#         split="validation",
#         task_start_token="<s_receipt-v1>",
#         prompt_end_token="<s_receipt-v1>",
#         sort_json_key=False,  # cord dataset is preprocessed, so no need for this
#     )

In [ ]:
added_tokens = train_dataset.get_token()

In [ ]:
len(added_tokens)
print("added_tokens", added_tokens)

In [ ]:
len(processor.tokenizer)
print(processor.tokenizer)

In [ ]:
processor.tokenizer.vocab_size

In [ ]:
pixel_values, labels, target_sequence = train_dataset[0]

In [ ]:
print(pixel_values.shape)

In [ ]:
for id in labels.tolist()[:30]:
    if id != -100:
        print(processor.decode([id]))
    else:
        print(id)

In [ ]:
print(target_sequence)

In [ ]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(
    ["<s_receipt-v1>"]
)[0]

In [ ]:
print("Pad token ID:", processor.decode([model.config.pad_token_id]))
print(
    "Decoder start token ID:", processor.decode([model.config.decoder_start_token_id])
)

Create PyTorch DataLoaders


In [38]:
from torch.utils.data import DataLoader
import torch

# feel free to increase the batch size if you have a lot of memory
# I'm fine-tuning on Colab and given the large image size, batch size > 1 is not feasible
# Set num_workers=4
multiprocessing_context = "fork" if torch.backends.mps.is_available() else None
train_dataloader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    # multiprocessing_context=multiprocessing_context,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    # multiprocessing_context=multiprocessing_context,
)

In [39]:
print("train dataloader", train_dataloader)
batch = next(iter(train_dataloader))
pixel_values, labels, target_sequences = batch
print(pixel_values.shape)

train dataloader <torch.utils.data.dataloader.DataLoader object at 0x1695a6aa0>
torch.Size([1, 3, 1280, 960])


In [40]:
for id in labels.squeeze().tolist()[:30]:
    if id != -100:
        print(processor.decode([id]))
    else:
        print(id)

<s_header>
<s_restaurant_name>
D
jour
nal
</s_restaurant_name>
<s_date>
22-
04
-20
24
13
:59
</s_date>
</s_header>
<s_items>
<s_item_qty>
1
</s_item_qty>
<s_item_name>
Salt
ed
Cara
mel
La
tte
</s_item_name>
<s_item_price>
4
9.000


In [41]:
print(len(train_dataset))
print(len(val_dataset))

12
2


In [42]:
batch = next(iter(val_dataloader))
pixel_values, labels, target_sequences = batch
print(pixel_values.shape)

torch.Size([1, 3, 1280, 960])


In [43]:
print(target_sequences[0])

<s_header><s_restaurant_name>Yakiniku Like Grand Indonesia</s_restaurant_name><s_date>27-01-2024 12:21</s_date></s_header><s_items><s_item_qty>1</s_item_qty><s_item_name>COUPLE SET 300GR</s_item_name><s_item_price>269.000</s_item_price><s_subitems><s_item_qty>1</s_item_qty><s_item_name>OCHA COLD (Free Refill)</s_item_name><s_item_price>15.000</s_item_price></s_subitems><sep/><s_item_qty>1</s_item_qty><s_item_name>OCHA HOT (Free Refill)</s_item_name><s_item_price>15.000</s_item_price></s_items><s_subtotal>299.000</s_subtotal><s_service>8.970</s_service><s_tax>30.797</s_tax><s_total>338.767</s_total></s>


Define LightingModule


In [44]:
from pathlib import Path
import re
from nltk import edit_distance
import numpy as np
import math
import torch

from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import LambdaLR

import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_only


class DonutModelPLModule(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        pixel_values, labels, _ = batch

        outputs = self.model(pixel_values, labels=labels)
        loss = outputs.loss
        self.log_dict({"train_loss": loss}, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        pixel_values, labels, answers = batch
        batch_size = pixel_values.shape[0]
        # we feed the prompt to the model
        decoder_input_ids = torch.full(
            (batch_size, 1),
            self.model.config.decoder_start_token_id,
            device=self.device,
        )

        outputs = self.model.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=max_length,
            early_stopping=True,
            pad_token_id=self.processor.tokenizer.pad_token_id,
            eos_token_id=self.processor.tokenizer.eos_token_id,
            use_cache=True,
            num_beams=1,
            bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
            return_dict_in_generate=True,
        )

        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(
                self.processor.tokenizer.pad_token, ""
            )
            seq = re.sub(
                r"<.*?>", "", seq, count=1
            ).strip()  # remove first task start token
            predictions.append(seq)

        scores = list()
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred)
            # NOT NEEDED ANYMORE
            # answer = re.sub(r"<.*?>", "", answer, count=1)
            answer = answer.replace(self.processor.tokenizer.eos_token, "")
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction: {pred}")
                print(f"    Answer: {answer}")
                print(f" Normed ED: {scores[0]}")

        return scores

    def validation_epoch_end(self, validation_step_outputs):
        # I set this to 1 manually
        # (previously set to len(self.config.dataset_name_or_paths))
        num_of_loaders = 1
        if num_of_loaders == 1:
            validation_step_outputs = [validation_step_outputs]
        assert len(validation_step_outputs) == num_of_loaders
        cnt = [0] * num_of_loaders
        total_metric = [0] * num_of_loaders
        val_metric = [0] * num_of_loaders
        for i, results in enumerate(validation_step_outputs):
            for scores in results:
                cnt[i] += len(scores)
                total_metric[i] += np.sum(scores)
            val_metric[i] = total_metric[i] / cnt[i]
            val_metric_name = f"val_metric_{i}th_dataset"
            self.log_dict({val_metric_name: val_metric[i]}, sync_dist=True)
        self.log_dict(
            {"val_metric": np.sum(total_metric) / np.sum(cnt)}, sync_dist=True
        )

    def configure_optimizers(self):
        # TODO add scheduler
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

Train


In [45]:
# Set epochs = 30
# Set num_training_samples_per_epoch = training set size
config = {
    "max_epochs": 30,
    "val_check_interval": 0.4,  # how many times we want to validate during an epoch
    "check_val_every_n_epoch": 1,
    "gradient_clip_val": 1.0,
    "num_training_samples_per_epoch": 425,
    "lr": 3e-5,
    "train_batch_sizes": [8],
    "val_batch_sizes": [1],
    # "seed":2022,
    "num_nodes": 1,
    "warmup_steps": 81,  # 425 / 8 = 54, 54 * 10 = 540, 540 * 0.15 = 81
    "result_path": "./result",
    "verbose": False,
}

model_module = DonutModelPLModule(config, processor, model)

In [50]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback

# wandb_logger = WandbLogger(project="Splitbill.id", name="receipt-parser-v1")


class PushToHubCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        print(f"Pushing model to the hub, epoch {trainer.current_epoch}")
        pl_module.model.push_to_hub(
            "mousserlane/sb-donut-receipt-parser-v1",
            commit_message=f"Training in progress, epoch {trainer.current_epoch}",
        )

    def on_train_end(self, trainer, pl_module):
        print(f"Pushing model to the hub after training")
        pl_module.processor.push_to_hub(
            "mousserlane/sb-donut-receipt-parser-v1", commit_message=f"Training done"
        )
        pl_module.model.push_to_hub(
            "mousserlane/sb-donut-receipt-parser-v1", commit_message=f"Training done"
        )


trainer = pl.Trainer(
    accelerator="cpu",
    devices=1,
    max_epochs=config.get("max_epochs"),
    val_check_interval=config.get("val_check_interval"),
    check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
    gradient_clip_val=config.get("gradient_clip_val"),
    # precision=16,  # we'll use mixed precision
    num_sanity_val_steps=0,
    # logger=wandb_logger,
    callbacks=[PushToHubCallback()],
)

trainer.fit(model_module)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type                      | Params
----------------------------------------------------
0 | model | VisionEncoderDecoderModel | 201 M 
----------------------------------------------------
201 M     Trainable params
0         Non-trainable params
201 M     Total params
807.535   Total estimated model params size (MB)


Epoch 0:  22%|██▏       | 4/18 [00:47<02:45, 11.80s/it, loss=12.3, v_num=12]

/Users/mousserlane/Codes/work/sb-sparrow-donut/.sb-sparrow-donut/lib/python3.10/site-packages/transformers/generation/utils.py:1338: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/Users/mousserlane/Codes/work/sb-sparrow-donut/.sb-sparrow-donut/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:641: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Epoch 0: 100%|██████████| 18/18 [03:23<00:00, 11.31s/it, loss=9.71, v_num=12]Pushing model to the hub, epoch 0


model.safetensors: 100%|██████████| 809M/809M [02:13<00:00, 6.05MB/s]


...

Epoch 29: 100%|██████████| 18/18 [05:29<00:00, 18.30s/it, loss=0.0695, v_num=12]

`Trainer.fit` stopped: `max_epochs=30` reached.


Pushing model to the hub after training


sentencepiece.bpe.model: 100%|██████████| 1.30M/1.30M [00:00<00:00, 2.43MB/s]
No files have been modified since last commit. Skipping to prevent empty commit.


Epoch 29: 100%|██████████| 18/18 [05:37<00:00, 18.73s/it, loss=0.0695, v_num=12]


Evaluation


In [51]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("mousserlane/sb-donut-receipt-parser-v1")
model = VisionEncoderDecoderModel.from_pretrained(
    "mousserlane/sb-donut-receipt-parser-v1"
)

In [52]:
import re
import json
import torch
from tqdm.auto import tqdm
import numpy as np

from donut import JSONParseEvaluator

from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()
model.to(device)

output_list = []
accs = []

dataset = load_dataset("mousserlane/id_receipt_dataset", split="test")

for idx, sample in tqdm(enumerate(dataset), total=len(dataset)):
    # prepare encoder inputs
    pixel_values = processor(
        sample["image"].convert("RGB"), return_tensors="pt"
    ).pixel_values
    pixel_values = pixel_values.to(device)
    # prepare decoder inputs
    task_prompt = "<s_receipt-v1>"
    decoder_input_ids = processor.tokenizer(
        task_prompt, add_special_tokens=False, return_tensors="pt"
    ).input_ids
    decoder_input_ids = decoder_input_ids.to(device)

    # autoregressively generate sequence
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # turn into JSON
    seq = processor.batch_decode(outputs.sequences)[0]
    seq = seq.replace(processor.tokenizer.eos_token, "").replace(
        processor.tokenizer.pad_token, ""
    )
    seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
    seq = processor.token2json(seq)

    ground_truth = json.loads(sample["ground_truth"])
    ground_truth = ground_truth["gt_parse"]
    evaluator = JSONParseEvaluator()
    score = evaluator.cal_acc(seq, ground_truth)

    accs.append(score)
    output_list.append(seq)

scores = {"accuracies": accs, "mean_accuracy": np.mean(accs)}
print(scores, f"length : {len(accs)}")

/Users/mousserlane/Codes/work/sb-sparrow-donut/.sb-sparrow-donut/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:641: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 4/4 [00:16<00:00,  4.15s/it]

{'accuracies': [0.06862745098039214, 1.0, 1.0, 0.752442996742671], 'mean_accuracy': 0.7052676119307657} length : 4


In [53]:
print("Mean accuracy:", np.mean(accs))

Mean accuracy: 0.7052676119307657
